In [43]:
from langchain_experimental.agents.agent_toolkits.pandas.base import create_pandas_dataframe_agent
from langchain_google_genai import ChatGoogleGenerativeAI

import streamlit as st
import pandas as pd
import matplotlib.pyplot as plt
from dotenv import load_dotenv

load_dotenv()

True

In [44]:
import sys
sys.path.append("C:/Users/Daniel/OneDrive - VNU-HCMUS/Desktop/DA_Project/data_analytics_llm")
from src.models.llms import load_llm

MODEL_NAME = "gemini-1.5-flash"

llm = load_llm(MODEL_NAME)

llm

ChatGoogleGenerativeAI(model='models/gemini-1.5-flash', google_api_key=SecretStr('**********'), temperature=0.0, max_output_tokens=1000, client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x000001AA488694E0>, default_metadata=())

In [45]:
from langchain.agents.agent_types import AgentType

In [53]:
df = pd.read_csv('../data/seattle-weather.csv')

agent = create_pandas_dataframe_agent(
    llm=llm,
    df=df,
    allow_dangerous_code=True,
    verbose=True,
    return_intermediate_steps=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
)

agent

AgentExecutor(verbose=True, agent=RunnableAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_log_to_str(x['intermediate_steps']))
})
| PromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={}, partial_variables={'df_head': '|    | date       |   precipitation |   temp_max |   temp_min |   wind | weather   |\n|---:|:-----------|----------------:|-----------:|-----------:|-------:|:----------|\n|  0 | 2012-01-01 |             0   |       12.8 |        5   |    4.7 | drizzle   |\n|  1 | 2012-01-02 |            10.9 |       10.6 |        2.8 |    4.5 | rain      |\n|  2 | 2012-01-03 |             0.8 |       11.7 |        7.2 |    2.3 | rain      |\n|  3 | 2012-01-04 |            20.3 |       12.2 |        5.6 |    4.7 | rain      |\n|  4 | 2012-01-05 |             1.3 |        8.9 |        2.8 |    6.1 | rain      |', 'tools': 'python_repl_ast - A Python shell. Use this to execute python commands. Input should be a valid python c

In [54]:
query = "Describe the data"
response = agent(query)



> Entering new AgentExecutor chain...
Thought:The head() method shows the first 5 rows of the dataframe.  I need to examine the data types of each column and look for any missing values to fully describe the data.
Action: python_repl_ast
Action Input: 
```python
print(df.info())
print(df.describe())
```<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1461 entries, 0 to 1460
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   date           1461 non-null   object 
 1   precipitation  1461 non-null   float64
 2   temp_max       1461 non-null   float64
 3   temp_min       1461 non-null   float64
 4   wind           1461 non-null   float64
 5   weather        1461 non-null   object 
dtypes: float64(4), object(2)
memory usage: 68.6+ KB
None
       precipitation     temp_max     temp_min         wind
count    1461.000000  1461.000000  1461.000000  1461.000000
mean        3.029432    16.439083     8.234771     3.24113

In [48]:
query = "describe the data by using python code"
response = agent(query)



> Entering new AgentExecutor chain...
Thought: I need to use pandas' `describe()` method to get descriptive statistics of the numerical columns in the dataframe.

Action: python_repl_ast
Action Input: 
```python
print(df.describe())
```          Unnamed: 0       Order ID  Quantity Ordered     Price Each  \
count  185950.000000  185950.000000     185950.000000  185950.000000   
mean     8340.388475  230417.569379          1.124383     184.399735   
std      5450.554093   51512.737110          0.442793     332.731330   
min         0.000000  141234.000000          1.000000       2.990000   
25%      3894.000000  185831.250000          1.000000      11.950000   
50%      7786.000000  230367.500000          1.000000      14.950000   
75%     11872.000000  275035.750000          1.000000     150.000000   
max     25116.000000  319670.000000          9.000000    1700.000000   

               Month          Sales           Hour  
count  185950.000000  185950.000000  185950.000000  
mean   

In [49]:
for step in response["intermediate_steps"]:
    print(step[1], "\n")

          Unnamed: 0       Order ID  Quantity Ordered     Price Each  \
count  185950.000000  185950.000000     185950.000000  185950.000000   
mean     8340.388475  230417.569379          1.124383     184.399735   
std      5450.554093   51512.737110          0.442793     332.731330   
min         0.000000  141234.000000          1.000000       2.990000   
25%      3894.000000  185831.250000          1.000000      11.950000   
50%      7786.000000  230367.500000          1.000000      14.950000   
75%     11872.000000  275035.750000          1.000000     150.000000   
max     25116.000000  319670.000000          9.000000    1700.000000   

               Month          Sales           Hour  
count  185950.000000  185950.000000  185950.000000  
mean        7.059140     185.490917      14.413305  
std         3.502996     332.919771       5.423416  
min         1.000000       2.990000       0.000000  
25%         4.000000      11.950000      11.000000  
50%         7.000000      14.9500

In [50]:
print(response["intermediate_steps"][0][0].tool_input.strip("```python").strip("```"))


print(df.describe())



In [51]:
#
if "intermediate_steps" in response:
    for step in response["intermediate_steps"]:
        if step[0].tool == "python_repl_ast":  # Chỉ lấy code từ tool Python
            code = step[0].tool_input.strip("```python").strip("```")  # Lấy nội dung action_input và loại bỏ các ký tự không hợp lệ
            print("Extracted Python Code:\n", code)  # In ra để kiểm tra

            # Thực thi mã Python
            exec(code)

Extracted Python Code:
 
print(df.describe())

          Unnamed: 0       Order ID  Quantity Ordered     Price Each  \
count  185950.000000  185950.000000     185950.000000  185950.000000   
mean     8340.388475  230417.569379          1.124383     184.399735   
std      5450.554093   51512.737110          0.442793     332.731330   
min         0.000000  141234.000000          1.000000       2.990000   
25%      3894.000000  185831.250000          1.000000      11.950000   
50%      7786.000000  230367.500000          1.000000      14.950000   
75%     11872.000000  275035.750000          1.000000     150.000000   
max     25116.000000  319670.000000          9.000000    1700.000000   

               Month          Sales           Hour  
count  185950.000000  185950.000000  185950.000000  
mean        7.059140     185.490917      14.413305  
std         3.502996     332.919771       5.423416  
min         1.000000       2.990000       0.000000  
25%         4.000000      11.950000    

In [52]:
#
if "intermediate_steps" in response:
    for step in response["intermediate_steps"]:
        if step[0].tool == "python_repl_ast":  # Chỉ lấy code từ tool Python
            code = step[0].tool_input.strip("```python").strip("```")  # Lấy nội dung action_input và loại bỏ các ký tự không hợp lệ
            print("Extracted Python Code:\n", code)  # In ra để kiểm tra

            # Thực thi mã Python
            exec(code)

Extracted Python Code:
 
print(df.describe())

          Unnamed: 0       Order ID  Quantity Ordered     Price Each  \
count  185950.000000  185950.000000     185950.000000  185950.000000   
mean     8340.388475  230417.569379          1.124383     184.399735   
std      5450.554093   51512.737110          0.442793     332.731330   
min         0.000000  141234.000000          1.000000       2.990000   
25%      3894.000000  185831.250000          1.000000      11.950000   
50%      7786.000000  230367.500000          1.000000      14.950000   
75%     11872.000000  275035.750000          1.000000     150.000000   
max     25116.000000  319670.000000          9.000000    1700.000000   

               Month          Sales           Hour  
count  185950.000000  185950.000000  185950.000000  
mean        7.059140     185.490917      14.413305  
std         3.502996     332.919771       5.423416  
min         1.000000       2.990000       0.000000  
25%         4.000000      11.950000    